In [2]:
import numpy as np

images = np.array([
    [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25]
    ],

    [1, 2, 3, 4, 5, 6, 7, 8, 9],
    [10, 11, 12, 13, 14, 15, 16, 17, 18],
    [19, 20, 21, 22, 23, 24, 25, 26, 27],
    [28, 29, 30, 31, 32, 33, 34, 35, 36],
    [37, 38, 39, 40, 41, 42, 43, 44, 45]
])

box_ind = np.array([1, 2, 0, 0])
boxes = np.array([
    []
])


_, image_depth, image_height, image_width, _ = images.shape
images_by_indices = images[box_ind]
boxes_unnormalized = boxes[:, ] * [
    image_depth - 1,
    image_height - 1,
    image_width - 1,
    image_depth - 1,
    image_height - 1,
    image_width - 1
]

array([[10, 11, 12, 13, 14, 15, 16, 17, 18],
       [19, 20, 21, 22, 23, 24, 25, 26, 27],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9]])

In [ ]:
def crop_and_resize(
    image,
    boxes,
    box_ind=None,
    crop_size=None,
    method='bilinear',
    extrapolation_value=0,
    name=None,
    box_indices=None):

    pass











